# Structured math experiments with Gemini
Kaleb Kassaw | 21 May 2024

In [1]:
import textwrap

import yaml
import json
import time

#import google.generativeai as genai
import gemini

from IPython.display import display, Markdown

def to_markdown(text):
    return Markdown(textwrap.indent(text.replace('•', '  *'), '> ', predicate=lambda _: True))

# This key is gitignored, because I didn't want to put it on the Internet. Find in Box.
with open('gemini-api.yaml') as rf:
    API_KEY = yaml.safe_load(rf)['api-key']

/opt/anaconda3/envs/gemini/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
standard_message = f'''\nPlease format your response as a Python dictionary modeled after the following:\n
{{
    "steps": <STEPS>,
    "answer": <ANSWER>
}}\n
with <STEPS> replaced by steps taken to get to the answer and <ANSWER> replaced with the final answer in simplified form.'''

annotations_calc = '../data-gen-tools/annotations/calc_basic.json'
annotations_elem_math = '../data-gen-tools/annotations/elementary_math.json'

class rate_limit:
    def __init__(self, n=14, per=60):
        self.n = n
        self.i = 0
        self.per = per

    def __call__(self):
        self.i += 1
        if self.i == self.n:
            self.__wait__()

    def __wait__(self):
        print('Cooling down. Rate limited for %i seconds...' % self.per)
        time.sleep(self.per)
        self.i = 0

def safe_json_loads(s):
    try:
        j = json.loads(s)
    except json.JSONDecodeError:
        j = None
    finally:
        return j

rl = rate_limit(n=59, per=60)
for ann in [annotations_calc, annotations_elem_math]:
    responses = []

    with open(ann, 'r') as rf:
        dict = json.load(rf)
        for x in dict['annotations']:
            chat_obj = gemini.Chat()
            try:
                response1 = chat_obj.send_message(x['prompt'] + standard_message)
                rl()
                responses.append(json.loads(response1.text.split('```python')[-1].split('```')[0]))
            except:
                rl.__wait__()
                response1 = chat_obj.send_message(x['prompt'] + standard_message)
                rl()
                responses.append(json.loads(response1.text.split('```python')[-1].split('```')[0]))

    with open(ann.replace('../data-gen-tools/annotations/', './answers/'), 'w') as wf:
        json.dump(responses, wf)

Cooling down. Rate limited for 60 seconds...
Cooling down. Rate limited for 60 seconds...
Cooling down. Rate limited for 60 seconds...


JSONDecodeError: Expecting ',' delimiter: line 47 column 15 (char 1586)

In [ ]:
print(responses[0])


{
    "steps": [
        "Subtract 10 from 16."
    ],
    "answer": 6
}

